In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime 
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
mpl.rcParams['figure.figsize']=(16,9)
pd.set_option('display.max_rows', 500)
sns.set(style="darkgrid")

In [ ]:
df_plot = pd.read_csv("processed.csv", sep = ';')
df_plot.sort_values('Date', ascending=False).head()

In [ ]:
fig = df_plot.iloc[15:,:].set_index('Date').plot()
fig.set_ylim(1,300000)
fig.set_yscale('log')   #Log scale is used for better visualization of exponential graph
fig.set_ylabel('Confirmed cases (source: John hopkins csse) (log scale)')

In [ ]:
import plotly.graph_objects as go
import plotly

In [ ]:
country_list = ['US', 'Italy', 'Spain', 'Germany', 'Korea,South']

In [ ]:
fig = go.Figure()

# Define how to plot individual traces
for i in country_list:
    fig.add_trace(go.Scatter(x = df_plot['Date'], 
                             y = df_plot[i],
                             name= i,
                             mode='markers+lines',
                             line_width = 1,
                             marker_size = 3
                            )
                 )

# Defines the overall graph properties    
fig.update_layout(xaxis_title='Time', 
                  yaxis_title='Confirmed infected people (source: John hopkins csse) (log scale)',
                  width = 1000, 
                  height = 800)
fig.update_yaxes(type = 'log')

fig.update_layout(xaxis_rangeslider_visible=True)

In [ ]:
import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()

# Defines layout of the dashboard
app.layout = html.Div([
    
    html.Label('Multi-Select Country'),
   
    dcc.Dropdown(
        id='country_drop_down',
        options=[
            {'label': 'Italy', 'value': 'Italy'},
            {'label': 'US', 'value': 'US'},
            {'label': 'Spain', 'value': 'Spain'},
            {'label': 'Germany', 'value': 'Germany'},
            {'label': 'Korea,South', 'value': 'Korea,South'}
        ],
        value=['US', 'Germany'], # which are pre-selected
        multi=True
    ),   
        
    dcc.Graph(figure=fig, id='main_window_slope')
])

In [ ]:
from dash.dependencies import Input, Output

#Make dashboard interactive
@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])
def update_figure(country_list):
    traces = [] 
    for each in country_list:
        traces.append(dict(x=df_plot.Date,
                                y=df_plot[each],
                                mode='markers+lines',
                                opacity=0.8,
                                line_width=1,
                                marker_size=3, 
                                name=each
                        )
                )
        
    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,
                yaxis_title="Confirmed infected people (source johns hopkins csse, log-scale)",
                xaxis={'title':'Timeline',
                        'tickangle':-30,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                        
                      },
                yaxis={'title':'Confirmed infected people (source johns hopkins csse, log-scale)',
                       'type':"log",
                       'range':'[1.1,5.5]'
                      }
        )
    }

In [ ]:
#app.run_server(debug=True, use_reloader=False)

In [ ]:
df_plot = pd.read_csv("processed.csv", sep  =';')
df_plot.sort_values('Date',ascending=False).head()

In [ ]:
V_url = "https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_doses_admin_global.csv"
df_Vacc = pd.read_csv(V_url)

In [ ]:
country_list = ['US', 'Italy', 'Spain', 'Germany', 'Korea,South']

In [ ]:
df_Iplot = df_plot
for country in country_list:
    df_Iplot[ country + " % Infected"] = 100*(df_plot[country])/(df_Vacc[df_Vacc['Country_Region']== country]['Population'].max())

In [ ]:
df_Iplot

In [ ]:
!pip install --user missingno

In [ ]:
import missingno as msno

In [ ]:
msno.matrix(df_Iplot)

In [ ]:
df_Iplot.interpolate(method = 'linear', limit_direction = 'backward', inplace = True)
msno.matrix(df_Iplot)

In [ ]:
fig = go.Figure()

# definition of plotting individual trace
for each in country_list:
    fig.add_trace(go.Scatter(x = df_Iplot.Date, 
                             y = df_Iplot[each+" % Infected"], 
                             mode = 'markers+lines',
                             opacity =0.9,
                             name =each, 
                             line_width = 2, 
                             marker_size =4
                            )
                 )
    
# layout inputs
fig.update_layout(xaxis_title = "Time", 
                  yaxis_title = "Percentage_of_Infected_Population (source johns hopkins csse, log-scale",
                  width = 1000, 
                  height = 500)

# fig.update_yaxes(type = 'log', range = [1.1, 9])
# fig.update_layout(xaxis_rangeslider_visible = True)
#fig.show(renderer = 'chrome')
fig.show()

In [ ]:
import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()

# Defines layout of the dashboard
app.layout = html.Div([
    
    html.Label('Multi-Select Country'),
   
    dcc.Dropdown(
        id='country_drop_down',
        options=[
            {'label': 'Italy', 'value': 'Italy'},
            {'label': 'US', 'value': 'US'},
            {'label': 'Spain', 'value': 'Spain'},
            {'label': 'Germany', 'value': 'Germany'},
            {'label': 'Korea,South', 'value': 'Korea,South'}
        ],
        value=['US', 'Germany'], # which are pre-selected
        multi=True
    ),   
        
    dcc.Graph(figure=fig, id='main_window_slope')
])

In [ ]:
from dash.dependencies import Input, Output

#Make dashboard interactive
@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])
def update_figure(country_list):
    traces = [] 
    for each in country_list:
        traces.append(dict(x=df_Iplot.Date,
                                y=df_Iplot[each+" % Infected"],
                                mode='markers+lines',
                                opacity=0.8,
                                line_width=1,
                                marker_size=3, 
                                name=each
                        )
                )
        
    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,
                yaxis_title="Percentage of Infected people (source johns hopkins csse, log-scale)",
                xaxis={'title':'Timeline',
                        'tickangle':-30,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                        
                      },
                yaxis={'title':'Percentage of Infected people (source johns hopkins csse, log-scale)',
                       'type':"log",
                       'range':'[1.1,5.5]'
                      }
        )
    }

In [ ]:
app.run_server(debug=True, use_reloader=False)